In [1]:
from pykrx import stock
from datetime import date
import datetime
import pandas as pd
import numpy as np
import calendar

### mm월에 대해 31일치가 모두 채워지도록 함수 생성
30일(ex 4월)이나 28일(2월)이어도 len이 31이 되도록 맨 마지막 값을 반복적으로 row로 채워줌

In [2]:
def daily_rtn(df):
    daily_rtn = pd.DataFrame(df['종가'].pct_change(1)).rename(columns = {'종가':'daily_rtn'})
    return daily_rtn

In [3]:
# 각 year당 month의 값 출력
def df_mth_list(yy,df):
    mth = df.loc['{}'.format(yy)].index.month.unique().tolist()
    for idx,i in enumerate(mth):
        if mth[idx] <10:
            mth[idx] = '0'+str(mth[idx])
        else:
            mth[idx] = str(mth[idx])
    return mth

In [4]:
# 01-31일까지 full month만들기(거래일 없는 날은 nan으로 채워서)
def get_full_month(yy,mm,df):
    # 01-31일까지 full month만들기
    temp_ = pd.DataFrame(data = [np.nan]*len(pd.date_range("{}-{}-01".format(yy,mm),"{}-{}-{}".format(yy,mm,calendar.monthrange(int(yy),int(mm))[1]))),
                         index = pd.date_range("{}-{}-01".format(yy,mm),"{}-{}-{}".format(yy,mm,calendar.monthrange(int(yy),int(mm))[1])),
                         columns = ['temp'])
    temp_ = temp_.reset_index()
    temp_ = temp_.rename(columns = {'index': '날짜'})


    df = df.loc['{}-{}'.format(yy,mm)]

    df_month = temp_.merge(df,on = '날짜',how = 'left')
    df_month.set_index('날짜')
    df_month = df_month.set_index('날짜').drop('temp',axis = 1).ffill()
    df_month = df_month.bfill() # 앞 결측치는 뒤의 값으로 

    for i in range(31-len(df_month)):
        tmp = pd.DataFrame(data = df_month.iloc[-1]).T
        df_month = pd.concat([df_month,tmp])

    # 현재 날짜보다 넘어가는 건 31일까지가아닌 현재날짜까지로 제거함 
    if df_month.index[-1] > df.index[-1]:
        df_month = df_month[: df.index[-1].date().isoformat()]
    
    return df_month

In [14]:
# 각 일자별 월 수익률
# ex) 20210101의 수익률은 ((2021-02-01 종가)-(2021-01-01 종가))/(2021-01-01 종가)
# input:[df_year_list, df]
def get_full_year_month_rtn(year,df):
    for yidx, yy in enumerate(year):
        mth = df_mth_list(year[yidx],df)
        
        for idx, mm in enumerate(mth):
            month = get_full_month(yy,mm,df)
            if idx == 0:
                one_year_full_mth = month
            else:
                one_year_full_mth = pd.concat([one_year_full_mth,month])
        
        if yidx == 0:
            total_month = one_year_full_mth
        else:
            total_month = pd.concat([total_month,one_year_full_mth])

#     shift를 사용해야할까 말아야할까?
#     total_month['mtn_rtn'] = total_month.pct_change(periods = 31).shift(-31)['종가']
    total_month['mtn_rtn'] = total_month.pct_change(periods = 31)['종가']
    return pd.DataFrame(total_month['mtn_rtn'])
# df 각 달의 1~31(31일이 안되는 경우 len =31까지 마지막 행 dup)을 채워넣음

In [17]:
# 각 일자별 분기 수익률
# ex) 20210101의 수익률은 ((2021-03-01 종가)-(2021-01-01 종가))/(2021-01-01 종가)
# input:[df_year_list, df]
def get_full_year_quarter_rtn(year,df):
    for yidx, yy in enumerate(year):
        mth = df_mth_list(year[yidx],df)
        
        for idx, mm in enumerate(mth):
            month = get_full_month(yy,mm,df)
            if idx == 0:
                one_year_full_mth = month
            else:
                one_year_full_mth = pd.concat([one_year_full_mth,month])
        
        if yidx == 0:
            total_month = one_year_full_mth
        else:
            total_month = pd.concat([total_month,one_year_full_mth])
    
#     total_month['quarter_rtn'] = total_month.pct_change(periods = 31*2).shift(-31*2)['종가']
    total_month['quarter_rtn'] = total_month.pct_change(periods = 31*2)['종가']
    return pd.DataFrame(total_month['quarter_rtn'])
# df 각 달의 1~31(31일이 안되는 경우 len =31까지 마지막 행 dup)을 채워넣음

In [7]:
df = stock.get_market_ohlcv("20220101", '20221231',"005930")

In [8]:
# df의 산출 년이 몇년인지 , 지금은 2021
year = list(map(str, df.index.year.unique().tolist()))

In [9]:
# 일별 수익률
daily_rtn = daily_rtn(df)
daily_rtn

,daily_rtn
날짜,
2022-01-03,NaN
2022-01-04,0.001272
2022-01-05,-0.016518
2022-01-06,-0.006460
2022-01-07,0.018205
...,...
2022-09-14,-0.022375
2022-09-15,-0.014085
2022-09-16,0.003571


In [20]:
# 월별 수익률
month_rtn = get_full_year_month_rtn(year,df)
month_rtn.tail(30) 

,mtn_rtn
2022-08-22,-0.008264
2022-08-23,-0.043689
2022-08-24,-0.037520
2022-08-25,-0.026101
2022-08-26,-0.021207
2022-08-27,-0.018003
2022-08-28,-0.027553
2022-08-29,-0.051780
2022-08-30,-0.050081
2022-08-31,-0.027687


In [18]:
# 분기별 수익률
quarter_rtn = get_full_year_quarter_rtn(year,df)
quarter_rtn

,quarter_rtn
2022-01-01,NaN
2022-01-02,NaN
2022-01-03,NaN
2022-01-04,NaN
2022-01-05,NaN
...,...
2022-09-16,-0.022609
2022-09-17,-0.063333
2022-09-18,-0.063333
2022-09-19,-0.060000
